# Genome scale model reconstruction of the smut fungus *Ustilago maydis*: Reproduction of simulations with CobraPy Notebook

## Introduction

This notebook is an extension to the article by Liebal et al., (2022) and reproduces the comparison of experimental growth rates with predictions by the genome scale metabolic model (GSMM) of *Ustilago maydis*. Finaly, the Memote quality check is performed. 

## Requirements
Make sure that the required files are present in the notebook directory. 

Files:
 * ../model/iCL1079.xml: SBML file, Genome scale metabolic model of *U. maydis*
 
Dependencies:
 * CPython 3.7.6
 * IPython 7.12.0
 * ipywidgets 7.5.1
 * matplotlib 3.1.3
 * numpy 1.18.1
 * pandas 1.0.1
 * cobra 0.17.1
 * memote 0.10.2

## Set-up compute environment 

In the following cell the python environment is set up. If CobraPy and Memote are not already downloaded, use the following command to download the packages them from the cell:
!{sys.executable} -m pip install cobra

In [ ]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
import os
# import xlrd
import pandas as pd
# import lxml
# for export of biolog substrate test dictionary
import csv
import itertools
import operator


# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

from Bio import SeqIO

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

# from escher import Builder

# loading Memote, quality assessment of GSMM
# from memote import test_model, snapshot_report
# from FastaTools import *
from iambcodes.cobra import *

## Support functions

In [ ]:
def TestSubstrate(model, EX_Sub, Rate=10):
    '''
    Function for testing substrate consumption with exchange reaction.
    '''
    with model:
        medium = model.medium
        medium[EX_Sub] = Rate
        medium['EX_co2_e'] = 0
        medium['EX_glc__D_e'] = 0
        model.medium = medium
        return round(model.slim_optimize(),2)

def MetNewFromExist(MetIn):
    Metout = Metabolite('{}_e'.format(MetIn.id[:-2]),
                       formula = MetIn.formula,
                       name = MetIn.name,
                       compartment = 'e')
    return Metout

def CreateTransReact(MetIn):
    reaction = Reaction('Trans_{}'.format(MetIn.id),
                       name = 'Transport c<->e {}'.format(MetIn.name),
                       lower_bound = -1000,
                       upper_bound = 1000)
    MetOut = MetNewFromExist(MetIn)
    reaction.add_metabolites({MetIn:-1.0, MetOut:1.0})
    
    return reaction

def CSVexport(myDict, FName='BiologGrowthTest'):
    fields = ['id','name','id_e','id_c','growth','CL_Growth']
    with open(FName, 'w') as f:
        w = csv.DictWriter(f, fields)
        w.writeheader()
        for k in myDict:
            w.writerow({field: myDict[k].get(field) or k for field in fields})

## Data loading

Now, the external data is integrated. The excel-sheet is parsed and the GSMM file is converted to a model variable.

In [ ]:
myxls = os.path.join('..','data','MSc-Lieven_Christian','sorted_growth_study.xlsx')
x1 = pd.ExcelFile(myxls)
df1 = x1.parse('Sheet1')
df1.head(3)

Loading the pan genome

In [ ]:
myECFile = os.path.join('..','data','Sequencing','EC_Annotation.txt')
EC_dict = extractEC(myECFile)
len(EC_dict)

## Model loading

In [ ]:
ModelFile = os.path.join('..','model','iCL1079.xml') # iCL1079.xml
model=read_sbml_model(ModelFile)
model.metabolites

In [ ]:
model.reactions.get_by_id('RXN-12710')

In [ ]:
# extracting all substrates from biolog list with BiGG metabolite identifiers
#  for source in df1['BiGG_ID'].astype(str) if source != 'nan']    
Substrate_dict = dict()
# Substrate_e = dict()
# Substrate_c = dict()
# Substrate_no = list()
for indx, Sub in enumerate(df1['BiGG_ID'].astype(str)):
    Sub_e = '{}_e'.format(Sub)
    Sub_c = '{}_c'.format(Sub)
    Sub_eIdx = np.where([Sub_e==met.id for met in model.metabolites])[0]
    Sub_cIdx = np.where([Sub_c==met.id for met in model.metabolites])[0]
    CLGrowth = str(round(df1['In_Silico_Growth_Rate'][indx],2))
    if Sub_cIdx.size>0 and Sub_eIdx.size>0:
        Substrate_dict[Sub] = {'name': model.metabolites[Sub_cIdx[0]].name, 'id_c': model.metabolites[Sub_cIdx[0]].id, 'id_e': model.metabolites[Sub_eIdx[0]].id, 'CL_Growth': CLGrowth}
    if Sub_cIdx.size>0 and Sub_eIdx.size<1:
        Substrate_dict[Sub] = {'name': model.metabolites[Sub_cIdx[0]].name, 'id_c': model.metabolites[Sub_cIdx[0]].id, 'id_e': None, 'CL_Growth': CLGrowth}
    if Sub_cIdx.size<1 and Sub_eIdx.size>0:
        Substrate_dict[Sub] = {'name': model.metabolites[Sub_eIdx[0]].name, 'id_c': None, 'id_e': model.metabolites[Sub_eIdx[0]].id, 'CL_Growth': CLGrowth}
    if Sub_cIdx.size<1 and Sub_eIdx.size<1:
        Substrate_dict[Sub] = {'name': None, 'id_c': None, 'id_e': None}


In [ ]:
len(Substrate_dict)

In [ ]:
#  change for loop to new substrate_dict with if loops to check which metabolic compartments exist

for Substrate in Substrate_dict.keys():
    if Substrate_dict[Substrate]['id_e'] is not None:
        EX_Sub = 'EX_{}'.format(Substrate_dict[Substrate]['id_e'])
        try: 
            model.reactions.get_by_id(EX_Sub)
        except KeyError:
            with model:
                model.add_boundary(model.metabolites.get_by_id(Substrate_dict[Substrate]['id_e']), type='exchange')
                myGrowth = TestSubstrate(model, EX_Sub)
                Substrate_dict[Substrate].update({'growth': str(myGrowth)})
#                 print('New exchange reaction for {} with growth: {}'.format(Substrate, myGrowth))        
        else:
            myGrowth = TestSubstrate(model, EX_Sub)
            Substrate_dict[Substrate].update({'growth': str(myGrowth)})
#             print('{} with growth: {}'.format(Substrate, myGrowth))
    elif Substrate_dict[Substrate]['id_c'] is not None:
        myMet = model.metabolites.get_by_id(Substrate_dict[Substrate]['id_c'])
        myTransport = CreateTransReact(myMet)
        EX_Sub = 'EX_{}_e'.format(Substrate)
        with model:
            model.add_reactions([myTransport])
            model.add_boundary(model.metabolites.get_by_id('{}_e'.format(Substrate)), type='exchange')
            myGrowth = TestSubstrate(model, EX_Sub)
            Substrate_dict[Substrate].update({'growth': str(myGrowth)})
#             print('New transport+exchange reaction for {} with growth: {}'.format(Substrate, myGrowth))        
#     else:
#         print('{} not in model.'.format(Substrate))
        
CSVexport(Substrate_dict)
print('Biolog-Model growth comparison exported.')

In [ ]:
# Export of gene list with locus tags
Genes_File = os.path.join('..','data','Uma_Genes.csv')
Genes = pd.DataFrame([[x.id,i.id] for x in model.genes for i in x.reactions], columns=['locus-tag', 'name'])
# Genes.to_csv(Genes_File, index=False)
# model.genes.get_by_id('UMAG_00518')
# model.reactions.get_by_id('ANTPPT')

In [ ]:
# EC_dict[model.genes.get_by_id('UMAG_00518').name]
Gene_common = [True if ModGene.name in EC_dict.keys() else False for ModGene in model.genes]
iUstiANDpan = [myGene.name for myGene in model.genes[Gene_common]]
iUstiNOTpan = [myGene.name for myGene in model.genes[list(map(operator.not_, Gene_common))]]
[EC_dict.pop(myGene.name, None) for myGene in model.genes]
panNOTiUsti = list(EC_dict.keys())
f1 = os.path.join('..','data','Sequencing','iUmaANDpan.txt')
f2 = os.path.join('..','data','Sequencing','iUmaNOTpan.txt')
f3 = os.path.join('..','data','Sequencing','panNOTiUma.txt')
with open(f1, "w") as f:
    f.write("\n".join(iUmaANDpan))
with open(f2, "w") as f:
    f.write("\n".join(iUmaNOTpan))
with open(f3, "w") as f:
    f.write("\n".join(panNOTiUma))
print('iUma in Pan:', sum(Gene_common))
print('iUma only:', len(model.genes)-sum(Gene_common))
print('Pan only:', len(panNOTiUma))
# model.genes.get_by_id('UMAG_00037')
# model.reactions.get_by_id('1.6.5.4-RXN')

### export of panNOTiUma list as fasta for KAAS analysis

In [ ]:
myFasta = 'pangenome_AA_Uma.faa'
myDir = os.path.join('..','data','Sequencing')
myFile = os.path.join(myDir,myFasta)

records = list()
with open(myFile) as infile:
    for record in SeqIO.parse(infile, 'fasta'):
        if record.id in panNOTiUsti:
            records.append(record)
with open(os.path.join(myDir,'panNOTiUsti.fasta'), 'w') as outfile:
    SeqIO.write(records, outfile, "fasta")

### Testing KEGG Biopython functions

In [ ]:
from Bio.KEGG import REST
result = REST.kegg_get('ko:k03800').read()
BRidx = result.find('BR:ko')
GOidx = result.find('map')
BRid = result[BRidx+5:BRidx+10]
GOid = result[GOidx+3:GOidx+8]
print(BRid, GOid)
print(result)

# finding multiple 'map' id in string of result
# https://stackoverflow.com/questions/47486563/how-can-i-get-index-of-two-of-more-duplicate-characters-in-a-string

In [ ]:
KAAStxt = 'KaasMap_panNOTiUsti.txt'
KAASpath = os.path.join('..','data','Sequencing','Analysis', KAAStxt)
myKAAS = pd.read_csv(KAASpath, delimiter='\t', names=['UMAG-ID','KO-ID'])
myKAAS
MAP_dict = dict()
for ko in myKAAS['KO-ID']:
    if str(ko) != 'nan':
        result = REST.kegg_get('ko:{}'.format(ko)).read()
        MAPidx = result.find('map')
        MAPid = result[MAPidx+3:MAPidx+8]
        if MAPid in MAP_dict:
            MAP_dict[MAPid]+=1
        else:
            MAP_dict[MAPid] = 1
MAP_dict

## Function definitions

We are testing three different growth conditions with substrates methanol, glycerol, and glucose. For each of the substrates, the corresponding exchange rate has to be activated, the approriate biomass composition formula has to be selected and exchange reactions for non-used substrates need to be closed.

## Simulation

Here, we start the simulation loop. For all substrates we use the substrate uptake rate and use the functions defined previously to update the model. The final growth rate is stored, and is then used in a plot versus the experimentally observed growth.

In [ ]:
model.reactions.query('ATP')
model.reactions.ATPS
# model.summary()

In [ ]:
# checking TCA consistency with objective to ATP hydrolysis/maintenance
with model as ATPM:
    ATPM.objective = 'ATPS'
    solution = ATPM.optimize()
    print(solution.fluxes['BIOMASS_REACTION'])
    print(model.summary())
    
# visualization    
builder = Builder()
# load U. maydis ESCHER map with Pentose phosphate pathway, Glycolysis

Escher_Central = os.path.join('Maps','umay_tcav7.json')
Escher_Glycine = os.path.join('Maps','umay_glycinev2.json')
builder = Builder(
    map_json=Escher_Central,
    model = model, # 'iCL1079.json',
)
# Run FBA with the model and add the flux data to the map
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes

builder.save_html('example_map.html')

## Comparison of BIOLOG and model

### Loading substrate list

In [ ]:
model.metabolites.get_by_id('gdpfuc_c')

In [ ]:
model.metabolites.query('Glucose')
myMets = pd.DataFrame([[x.name,x.id] for x in model.metabolites])
#     print('%9s : %s' % (x.id, x.formula))
# myMets
check = 'fuc'
matching = [idx for idx,s in enumerate(myMets[1]) if check in s]
myMets.loc[matching]

In [ ]:
model_loop = model.copy()
growth_simulated = [];
for index, row in df1.iterrows():
  print(index)
  if row['Substrate'] == 'Methanol':
    model_loop = AdaptMethanol(model_loop, row['uptake rate (mmol/gCDW/h)'])
  elif row['Substrate'] == 'Glucose':
    model_loop = AdaptGlucose(model_loop, row['uptake rate (mmol/gCDW/h)'])
  elif row['Substrate'] == 'Glycerol':
    model_loop = AdaptGlycerol(model_loop, row['uptake rate (mmol/gCDW/h)'])
  else:
    print('substrate not considered')      
  model_loop.optimize()
  growth_simulated.append(model_loop.objective.value)


## Visualization

In [ ]:
plt.rc('axes', labelsize=12) 
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10) 
plt.rc('legend', fontsize=12) 
# plt.figure(figsize=(4,3))
plt.scatter(df1['growth rate (/h)'][1:8], growth_simulated[1:8], s=100, c='k', marker='o', );
plt.scatter(df1['growth rate (/h)'][8], growth_simulated[8], s=100, c='k', marker='s');
plt.scatter(df1['growth rate (/h)'][0], growth_simulated[0], s=100, c='k', marker='x');
plt.xlabel('Experiment (/h)');
plt.ylabel('Simulation (/h)');
myline = np.linspace(0,np.max(growth_simulated),10);
plt.plot(myline,myline,'k--');
# plt.title('Growth rate comparison');
# correlation coefficient between experimental and simulated data
R2 = np.corrcoef(np.array([growth_simulated[0:9],df1['growth rate (/h)'][0:9]]))
R2Legend = r'Optimum, $R^2 = %.4f$'%(R2[0,1]) 
plt.legend([R2Legend,'Methanol (van Dijken)','Glycerol (deKoning, Moon)','Glucose (Lehnen)'], loc=2);
plt.style.use('seaborn-paper')
# plt.savefig('iOpol_growth_exp-vs-sim.png', bbox_inches='tight')
# plt.savefig('iOpol_growth_exp-vs-sim.pdf', bbox_inches='tight')

## Memote quality test

Memote performs quality assessments of genome scale metabolic models. The process can take a while, around 5 min.

In [ ]:
mem_return, mem_result = test_model(model, results=True)
with open('iUmay_MemoteReport.html', 'w') as f:
    f.write(snapshot_report(mem_result))



### Mass balance test

In [ ]:
import json

# myFile = os.path.join('..','iUL_versions','iUL959_200810.xml') # os.path.join('..','iUL_versions','frank','out_metadata.xml') # os.path.join('..','iUL_versions','iUL959_112718.xml')
# model = read_sbml_model(myFile)
mysol = [[reaction.id, reaction.reaction, reaction.check_mass_balance()] for reaction in model.reactions if reaction.check_mass_balance()]
print('unbalanced reactions: {}'.format(len(mysol)))
df_mysol = pd.DataFrame(mysol, columns=['Abbreviation','Formula','Misbalance'])

# deleting reactions with following strings
DelStr_all = ['Ex_', 'avg', 'BIOM', 'PROT', 'LIP', 'STER', 'DNA', 'RNA', 'CARB', 'GLYCE']
for DelStr in DelStr_all:
    indices = [i for i, s in enumerate(list(df_mysol['Abbreviation'])) if DelStr in s]
    df_mysol.drop(df_mysol.index[[indices]], inplace=True)
    
df_mysol.reset_index(inplace=True)
df_mysol.to_csv('iOpol_unbalanced.txt', index=False)

df_mysol


In [ ]:
model.optimize()
print(model.summary())
[reaction.flux for reaction in model.reactions.query('TRNA')]
# print(model.metabolites.get_by_id('fum[c]').summary())

In [ ]:
model.metabolites.query('tre_c')

### Dependencies and versions
The code below was used to identify the versions of dependent packages.

In [ ]:
%load_ext watermark
%watermark -v -m -p ipywidgets,matplotlib,numpy,pandas,cobra,memote,watermark